In [1]:
import os, sys
from keras import backend as K

%matplotlib inline

Using Theano backend.
Using gpu device 0: GeForce GTX 750 Ti (CNMeM is disabled, cuDNN 5103)
/home/manpun/anaconda2/lib/python2.7/site-packages/theano/sandbox/cuda/__init__.py:600: UserWarning: Your cuDNN version is more recent than the one Theano officially supports. If you see any problems, try updating Theano or downgrading cuDNN to version 5.
  warnings.warn(warn)


## First get the training, validation and sample data in order.

In [2]:
homeDir = os.getcwd()

In [3]:
#dataDir = homeDir + "/data/"
dataDir = homeDir + "/data/sample/"
train_path = dataDir + "train/"
valid_path = dataDir + "valid/"
model_path = homeDir + "/data/models/"

In [4]:
from utils import *
from Vgg16 import Vgg16

### Getting the data ready for processing.

In [ ]:
#trn_data = get_data(batches)
#val_data = get_data(val_batches)

In [ ]:
batches

## Get the vgg16 model and initialize it

In [ ]:
vgg = Vgg16()

In [ ]:
#Set constants. You can experiment with no_of_epochs to improve the model. You can reduce the batch_size 
#depending on the memory contraints of gpu
batch_size=8
no_of_epochs=3

In [ ]:
# get_batches function transforms images into arrays and the gets them in batches.

batches = vgg.get_batches(train_path, batch_size=batch_size)
val_batches = vgg.get_batches(valid_path, batch_size=batch_size)

In [ ]:
vgg.model.summary()

## Finetune 
Now I will remove the final dense layer for classifying in to 1000 categories and add a new layer for only cats and dogs.

In [ ]:
vgg.model.pop() #Remove the final layer
for layer in vgg.model.layers: layer.trainable=False # Set all other layers to untrainable

In [ ]:
vgg.model.add(Dense(2, activation='softmax')) 
#Adding a new dense layer wiht only 2 outputs and softmax acitvation as it is the output layer

In [ ]:
# Using different optimizers 
opt = RMSprop(lr=0.01)


In [ ]:
vgg.compile()
#vgg.model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])
vgg.fit(batches, val_batches, nb_epoch=1)

In [ ]:
vgg.model.save_weights(model_path+'finetune_last_layer.h5')

Now let us train all the dense layers and see the results

In [ ]:
layers= vgg.model.layers

In [ ]:
dense_layer_idx = [index for index,layer in enumerate(layers) if type(layer) is Dense]

In [ ]:
dense_layer_idx

In [ ]:
for l in dense_layer_idx:
    layers[l].trainable=True

In [ ]:
layers[34].trainable

In [ ]:
K.set_value(opt.lr, 0.01)
vgg.fit(batches, val_batches, nb_epoch=3)

In [ ]:
model.save_weights(model_path+'finetune_allDenseLayers.h5')

## It lloks like we are underfitting the model.
Let us modify the vgg model for our purpose

In [5]:
model = vgg_ft(2)

/home/manpun/anaconda2/lib/python2.7/site-packages/keras/layers/core.py:621: UserWarning: `output_shape` argument not specified for layer lambda_1 and cannot be automatically inferred with the Theano backend. Defaulting to output shape `(None, 3, 224, 224)` (same as input shape). If the expected output shape is different, specify it via the `output_shape` argument.
  .format(self.name, input_shape))


In [7]:
model.load_weights(model_path + 'finetune_last_layer.h5')

In [8]:
layers = model.layers

In [41]:
layers[-2].output_shape

(None, 4096)

In [9]:
lastConvIdx = [index for index,layer in enumerate(layers) if type(layer) is Convolution2D][-1]

Creating a seperate model for conv and fc layers

In [12]:
convLayers = layers[:lastConvIdx+1]
convModel = Sequential(convLayers)
fcLayers = layers[lastConvIdx+1:]

In [14]:
batch_size = 8

In [19]:
batches = get_batches(train_path, shuffle=False, batch_size = batch_size)
val_batches = get_batches(valid_path, shuffle=False, batch_size=batch_size)

val_classes = val_batches.classes
trn_classes = batches.classes
val_labels = onehot(val_classes)
trn_labels = onehot(trn_classes)

Found 200 images belonging to 2 classes.
Found 50 images belonging to 2 classes.


SAving the oupiut of convolution block so that the processing is not needed every time we fit .

In [20]:
batches.nb_sample

200

In [22]:
train_features = convModel.predict_generator(batches,batches.nb_sample)
val_features = convModel.predict_generator(val_batches, val_batches.nb_sample)

In [25]:
save_array(model_path + 'train_convlayer_features.bc', train_features)
save_array(model_path + 'valid_convlayer_features.bc', val_features)

In [26]:
trn_features = load_array(model_path+'train_convlayer_features.bc')
val_features = load_array(model_path+'valid_convlayer_features.bc')

In [28]:
val_features.shape

(50, 512, 14, 14)

In [33]:
def new_weights(layer):
    return [o/2 for o in layer.get_weights()]
opt = RMSprop(lr=0.001, rho=0.7)
def get_fc_model(p):
    model = Sequential([
        MaxPooling2D(input_shape = convLayers[-1].output_shape[1:]),
        Flatten(),
        Dense(4096,activation="relu"),
        Dropout(p),
        #BatchNormalization(),
        Dense(4096, activation='relu'),
        Dropout(p),
        #BatchNormalization(),
        Dense(2, activation='softmax')
    ])
    for l1,l2 in zip(model.layers, fcLayers): l1.set_weights(new_weights(l2))

    model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])
    return model

In [50]:
fcLayers[-1].get_weights()[1].shape

(1000,)

In [34]:
fc_model = get_fc_model(0)

ValueError: Layer weight shape (4096, 2) not compatible with provided weight shape (4096, 1000)